In [1]:
import numpy as np
import pandas as pd
import json
import imageio
import glob
import os
import cv2
import copy

## Flip images

In [2]:
def flip(old_coor, center):
    new_coors = []
    for coor in old_coor:
        if coor > center:
            new_coor = center - (coor - center)
            # check if new coordinate exceeds the image orders
            if new_coor >= center*2:
                new_coors.append(center*2-1)
            else:
                new_coors.append(new_coor)
        else:
            new_coor = center + (center - coor)
            # check if new coordinate exceeds the image orders
            if new_coor < 1:
                new_coors.append(1)
            else:
                new_coors.append(new_coor)
    return new_coors

In [3]:
# TODO change image directory and extension
# better keep images and annotations in the same folder
set_dir = 'train'
image_dir = os.path.join('/Users/yucelkenanozturk/Desktop/Internship/labelled images/', set_dir)
annotation_file = os.path.join(image_dir, 'via_region_data(9)_added16newimages.json')
image_extension = '.bmp'
flip_type = ['lr'] # or horizontal


# import images
image_paths = []
for file in os.listdir(image_dir):
    if file.endswith(image_extension):
        image_paths.append(file)

# import masks
masks = pd.DataFrame(json.load(open(annotation_file))).T
columns = masks.columns

# image center
center_x, center_y = 1024, 1024

#### vertical flip
if 'ud' in flip_type:
    
    new_filenames = []
    for image_path in image_paths:
        new_image_name = 'ud_' + image_path
        new_filenames.append(new_image_name)
        
        # read image, flip and write
        image = imageio.imread(os.path.join(image_dir, image_path))
        imageio.imwrite(new_image_name, np.flipud(image))

        all_regions = list(masks[masks['filename'] == image_path]['regions'].item().keys())
        for region_id in all_regions:
            old_y_coor = copy.deepcopy(masks[masks['filename'] == image_path]['regions'].item()[region_id]['shape_attributes']['all_points_y'])
            new_y_coor = flip(old_y_coor, center_y)
            masks[masks['filename'] == image_path]['regions'].item()[region_id]['shape_attributes']['all_points_y'] = new_y_coor


        masks.loc[masks['filename'] == image_path, 'filename'] = new_image_name

    new_filenames = [file + str(masks['size'][0]) for file in new_filenames]
    masks.index = new_filenames

    # write back the annotations
    with open('new_region_data_ud.json', 'w') as outfile:
        json.dump(masks.T.to_dict(), outfile)


#### horizontal flip
if 'lr' in flip_type:
    new_filenames = []
    for image_path in image_paths:
        new_image_name = 'lr_' + image_path
        new_filenames.append(new_image_name)
        
        # read image, flip and write
        image = imageio.imread(os.path.join(image_dir, image_path))
        imageio.imwrite(new_image_name, np.fliplr(image))

        all_regions = list(masks[masks['filename'] == image_path]['regions'].item().keys())
        for region_id in all_regions:
            old_x_coor = copy.deepcopy(masks[masks['filename'] == image_path]['regions'].item()[region_id]['shape_attributes']['all_points_x'])
            new_x_coor = flip(old_x_coor, center_x)
            masks[masks['filename'] == image_path]['regions'].item()[region_id]['shape_attributes']['all_points_x'] = new_x_coor


        masks.loc[masks['filename'] == image_path, 'filename'] = new_image_name

    new_filenames = [file + str(masks['size'][0]) for file in new_filenames]
    masks.index = new_filenames

    # write back the annotations
    with open('new_region_data_lr.json', 'w') as outfile:
        json.dump(masks.T.to_dict(), outfile)

## Rotate image

In [2]:
def rotate_point(x_coor, y_coor, radian, center_x, center_y):
    '''
    calculates the new coordinates of a point after rotation
    x_coor, y_coor: coordinates of the point to be rotated
    radian: degree of rotation in radians
    center_x, center_y: coordinates of rotation's center
    
    returns new coordinates of the point as integers
    '''
    new_x_coor = np.cos(radian)*(x_coor - center_x) - np.sin(radian)*(y_coor - center_y) + center_x
    new_y_coor = np.sin(radian)*(x_coor - center_x) + np.cos(radian)*(y_coor - center_y) + center_y
    
    # check if the new coordinates exceed the image boundries
    if new_x_coor >= center_x*2:
        new_x_coor = center_x*2 - 1
    elif new_x_coor < 1:
        new_x_coor = 1
    
    if new_y_coor >= center_y*2:
        new_y_coor = center_y*2 - 1
    elif new_y_coor < 1:
        new_y_coor = 1
        
    return int(round(new_x_coor)), int(round(new_y_coor))

def rotate_region(x_coors, y_coors, radian, center_x, center_y):
    '''
    calculates the new coordinates of a region after rotation
    x_coors, y_coors: coordinates of the region points to be rotated
    radian: degree of rotation in radians
    center_x, center_y: coordinates of rotation's center
    
    returns new coordinates of the region points as lists
    '''
    new_x_coors, new_y_coors = [], []
    for x_coor, y_coor in zip(x_coors, y_coors):
        new_x_coor, new_y_coor = rotate_point(x_coor, y_coor, radian, center_x, center_y)
        new_x_coors.append(new_x_coor)
        new_y_coors.append(new_y_coor)
    return new_x_coors, new_y_coors

def rotate_image_mask(image_mask, image_name, radian, center_x, center_y):
    '''
    calculates the new coordinates of a region after rotation
    inputs
    regions: all regions of to be rotated mask
    radian: degree of rotation in radians
    center_x, center_y: coordinates of rotation's center
    
    returns new annotation as a row to be appended to the main annotations dataframe
    '''
    # copy the dict
    new_mask = copy.deepcopy(image_mask)
    new_regions = new_mask['regions'].item()
    new_mask['filename'] = image_name
    for region_id in new_regions:
        subst_dict = {region_id: {
                                    'shape_attributes' : {
                                                            'all_points_x': [0],
                                                            'all_points_y': [0],
                                                            'name' : 'polygon'
                                                        },
                                    'region_attributes' : {
                                                            'component' : 'wire'
                                    }
        }}
        x_coors = new_regions[region_id]['shape_attributes']['all_points_x']
        y_coors = new_regions[region_id]['shape_attributes']['all_points_y']
        new_region_coor_x, new_region_coor_y = rotate_region(x_coors, y_coors, radian, center_x, center_y)
        subst_dict[region_id]['shape_attributes']['all_points_x'] = new_region_coor_x
        subst_dict[region_id]['shape_attributes']['all_points_y'] = new_region_coor_y
        new_mask['regions'].item()[region_id]['shape_attributes'] = subst_dict[region_id]['shape_attributes']
    new_mask.index = [image_name + str(image_mask['size'][0])]
    return new_mask

def rotateImage(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def rotate_img_and_mask(image, image_name, image_mask, radian):
    '''
    rotates image and its mask
    inputs
    image: as np.array
    radian: degree of rotation in radians
    regions: mask of given image
    
    returns rotated image as np.array and annotation as dataframe
    '''
    center_x = image.shape[0] // 2
    center_y = image.shape[1] // 2
    
    annotation_for_img = rotate_image_mask(image_mask, image_name, -radian, center_x, center_y)
    return rotateImage(image, radian * 180 / np.pi), annotation_for_img

In [4]:
# TODO change image directory and extension
# better keep images and annotations in the same folder
set_dir = 'train'
image_dir = os.path.join('/Users/yucelkenanozturk/Desktop/Internship/labelled images/flip_rotate/flipped_rotated', set_dir)
annotation_file = os.path.join(image_dir, 'via_region_data.json')
image_extension = '.bmp'
rotation_angles = [45, 90, 135, 180, 225, 270, 315]

# import images
image_paths = []
for file in os.listdir(image_dir):
    if file.endswith(image_extension):
        image_paths.append(file)

# import masks
masks = pd.DataFrame(json.load(open(annotation_file))).T

# image center
center_x, center_y = 1024, 1024

# perform image and mask rotation
angular_masks = pd.DataFrame()
for image_path in image_paths:
    image = imageio.imread(os.path.join(image_dir, image_path))
    for angle in rotation_angles:
        masks = pd.DataFrame(json.load(open(annotation_file))).T
        image_mask = copy.deepcopy(masks[masks['filename'] == image_path])
        new_image_name = str(angle) + '_' + image_path
        
#         for images of varying sizes uncomment below lines
#         center_x = image.shape[0] // 2
#         center_y = image.shape[1] // 2
        
        rotated_image, new_annotation = rotate_img_and_mask(image, new_image_name, 
                                                            image_mask, angle * np.pi /180)
        # write rotated image
        imageio.imwrite(new_image_name, rotated_image)
        #angular_masks = 
        angular_masks = angular_masks.append(new_annotation)
        
        # write back the annotations
        with open('new_region_data_{}.json'.format(angle), 'w') as outfile:
            json.dump(angular_masks.T.to_dict(), outfile)


        
#### WHAT TO DO WITH NEW_MASK_COOR? PROBABLY NEED TO CREATE NEW JSON BY ADDING ROWS TO THE ANNOTATION FILE
#### FIRST IMPORT THE MASKS THOUGH :D LOOK AT THE masks VARIABLE